In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import numpy as np
""" if(torch.cuda.is_available()):
    device = "cuda"
else:
    device = "cpu" """
device ="cpu"

In [9]:
data1=pd.read_csv("data/newbadtrb.csv")
data2=pd.read_csv("data/newbadtrg.csv")

data3=pd.read_csv("data/newbadvalb.csv")
data4=pd.read_csv("data/newbadvalg.csv")

data5=pd.read_csv("data/newgoodtr.csv")
data6=pd.read_csv("data/newgoodval.csv")

data = pd.concat([data1,data2,data3,data4,data5,data6])
data.head()

,Nose X,Nose Y,Nose Z,Left Shoulder X,Left Shoulder Y,Left Shoulder Z,Right Shoulder Z,Label
0,245.534248,-94.723091,-1327.650375,570.032234,20.433583,-779.603319,-756.481390,bad
1,211.337738,-195.927644,-1191.081390,556.738281,-158.444266,-591.870146,-609.588518,bad
2,210.084839,-187.959208,-1522.594185,546.887970,-138.855085,-787.883749,-791.414738,bad
3,232.364426,-182.437720,-1847.881851,564.786606,-113.810763,-1037.135983,-1001.266136,bad
4,257.091904,-140.993042,-1571.872673,521.389008,46.378870,-814.818363,-818.863306,bad


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
import torch.utils
from pickle import dump
encoder = LabelEncoder()
data["Label"] = encoder.fit_transform(data["Label"])
X=data.drop(columns="Label").to_numpy(dtype=np.float32)
y=data["Label"].to_numpy(dtype=np.float32)
#scaler=MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)

train_dataset= torch.utils.data.TensorDataset(torch.tensor(X_train),torch.tensor(y_train))

test_dataset= torch.utils.data.TensorDataset(torch.tensor(X_test),torch.tensor(y_test))
#dump(scaler, open('scaler.pkl', 'wb'))

train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=3,num_workers=3)
test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=1,num_workers=3)

In [11]:

class slouch_detection(nn.Module):
    def __init__(self):
        super(slouch_detection,self).__init__()
        self.layers = nn.Sequential(
            nn.LayerNorm(7),
            nn.Linear(7,36),
            nn.ReLU(),
            nn.Linear(36,15),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(15,12),
            nn.ReLU(),
            nn.Linear(12,6),
            nn.ReLU(),
            nn.Linear(6,3),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(3,1),
            nn.Sigmoid()
        )
    @torch.compile(fullgraph=True, dynamic=True, options = {"epilogue_fusion": True, "max_autotune": True,
                    "shape_padding": True})
    def forward(self,x):
        return self.layers(x)


In [12]:
model = slouch_detection().to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Assuming you're using Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # You can adjust the learning rate as needed


In [13]:
total=0
for params in model.parameters():
    total+=params.numel()
total

1152

In [14]:
def train_model(model, train_loader, optimizer, criterion, epochs,device,best_acc):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            labels=labels.unsqueeze(dim=1).to(device)
            outputs = model(inputs.to(device))
            #print(outputs.size(),labels.unsqueeze(dim=1).size())
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            
            # Calculate accuracy
            predicted = torch.round(outputs)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            print()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")
        test_model(model, test_dataloader, criterion,device,best_acc)

# Testing loop
def test_model(model, test_loader, criterion,device,best_acc):
    model.eval()
    with torch.inference_mode():
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            labels=labels.unsqueeze(dim=1).to(device)
            outputs = model(inputs.to(device))
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            predicted = torch.round(outputs)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        test_loss = running_loss / len(test_loader.dataset)
        test_acc = correct / total
        if(best_acc<test_acc):
            torch.save(model.state_dict(),"slouch_detector.pt")
            best_acc=test_acc
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


In [15]:
best_acc=0
train_model(model, train_dataloader, optimizer, criterion,device=device, epochs=50,best_acc=best_acc)  # Adjust epochs as needed

test_model(model, test_dataloader, criterion,device,best_acc)




























































































































































































































































































































































































































Epoch 1/50, Loss: 0.6374, Accuracy: 0.6989
Test Loss: 0.6367, Test Accuracy: 0.6722



























































































































































































































































































































































































































Epoch 2/50, Loss: 0.6065, Accuracy: 0.6989
Test Loss: 0.6137, Test Accuracy: 0.6722












In [16]:
test_model(model, test_dataloader, criterion,device,best_acc)

Test Loss: 0.2927, Test Accuracy: 0.8616
